In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/life-expectancy-cleaned-csv/life_expectancy_clean.csv


In [19]:
import pandas as pd
df = pd.read_csv("/kaggle/input/life-expectancy-cleaned-csv/life_expectancy_clean.csv")
# print(df.columns.tolist())
categories = [0,60,75,100]
labels = ['Low','Medium','High']
df['Life_expectancy_category']= pd.cut(df['Life expectancy '],bins=categories,labels=labels)
print(df[['Life_expectancy_category','Life expectancy ']].head())
print(df[['Life_expectancy_category','Life expectancy ']].tail())



  Life_expectancy_category  Life expectancy 
0                   Medium              65.0
1                      Low              59.9
2                      Low              59.9
3                      Low              59.5
4                      Low              59.2
     Life_expectancy_category  Life expectancy 
2933                      Low              44.3
2934                      Low              44.5
2935                      Low              44.8
2936                      Low              45.3
2937                      Low              46.0


In [20]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

df_num = df.select_dtypes(include=['float64','int64'])
X = df_num.drop(columns=['Life expectancy '],errors='ignore')
X = add_constant(X)

vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values,i) for i in range(X.shape[1])]
vif.sort_values(by="VIF",ascending=False)


,Feature,VIF
0,const,214857.095465
3,infant deaths,176.797867
9,under-five deaths,175.783874
17,thinness 5-9 years,8.861208
16,thinness 1-19 years,8.776145
14,GDP,5.218959
5,percentage expenditure,5.038948
19,Schooling,3.334651
18,Income composition of resources,3.069051
12,Diphtheria,2.217843


infant deaths coloumn and under-five deaths are abnormally highly multicollinear.
so infant death column is removed before modelling.

In [21]:
#Strip extra spaces from column names
df.columns = df.columns.str.strip()

print(df.columns.tolist())

#Now use the cleaned column names
df['Thinness Average'] = df[['thinness  1-19 years', 'thinness 5-9 years']].mean(axis=1)

#Also remove the infants and both thinness column
df.drop(columns=['infant deaths','thinness  1-19 years','thinness 5-9 years','percentage expenditure'], inplace=True)

df.head()

['Country', 'Year', 'Status', 'Life expectancy', 'Adult Mortality', 'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B', 'Measles', 'BMI', 'under-five deaths', 'Polio', 'Total expenditure', 'Diphtheria', 'HIV/AIDS', 'GDP', 'Population', 'thinness  1-19 years', 'thinness 5-9 years', 'Income composition of resources', 'Schooling', 'Life_expectancy_category']


,Country,Year,Status,Life expectancy,Adult Mortality,Alcohol,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,Income composition of resources,Schooling,Life_expectancy_category,Thinness Average
0,Afghanistan,2015,Developing,65.0,263.0,0.01,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,0.479,10.1,Medium,17.25
1,Afghanistan,2014,Developing,59.9,271.0,0.01,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,0.476,10.0,Low,17.50
2,Afghanistan,2013,Developing,59.9,268.0,0.01,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,0.470,9.9,Low,17.70
3,Afghanistan,2012,Developing,59.5,272.0,0.01,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,0.463,9.8,Low,17.95
4,Afghanistan,2011,Developing,59.2,275.0,0.01,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,0.454,9.5,Low,18.20


In [22]:
#import libraries for chi-square test
from scipy.stats import chi2_contingency

#create a merged and counted table with both the features
table = pd.crosstab(df['Status'], df['Life_expectancy_category'])

chi2, p, dof, expected = chi2_contingency(table)
print("Chi-square statistic:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("\nContingency Table:\n", table)

Chi-square statistic: 918.8748102038433
p-value: 2.943541643170323e-200
Degrees of freedom: 2

Contingency Table:
 Life_expectancy_category  Low  Medium  High
Status                                     
Developed                   0      90   422
Developing                594    1430   402


There is a statistically significant association between a country's development status (Developed or Developing) and its life expectancy category (Low, Medium, High). As the p-value is way too smaller than 0.05 .

Developed countries:

0 countries in the Low life expectancy category.

Majority (≈ 82%) are in the High category.

Very few fall in the Medium category.


Developing countries:

A significant number (≈ 24%) are in the Low category.

Most (≈ 59%) fall in the Medium category.

Far fewer reach the High life expectancy level compared to developed nations.



In [23]:
#encode categorical variables
from sklearn.preprocessing import OneHotEncoder
df['Status_encoded'] = df['Status'].map({'Developing': 0, 'Developed': 1})

df['Country_encoded'] = df.groupby('Country')['Life expectancy'].transform('mean')

year_array = df[['Year']].values
encoder = OneHotEncoder(sparse=False, drop=None)
year_encoded = encoder.fit_transform(year_array)
year_encoded_df = pd.DataFrame(year_encoded, columns=encoder.get_feature_names_out(['Year']))
df = pd.concat([df.drop(columns=['Year']), year_encoded_df], axis=1)

#drop the unnecessary columns
df.drop(columns=['Status','Country','Life_expectancy_category'], inplace=True)

df[['Status_encoded','Country_encoded','Year_2000']].head()

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Status_encoded,Country_encoded,Year_2000
0,0,58.19375,0.0
1,0,58.19375,0.0
2,0,58.19375,0.0
3,0,58.19375,0.0
4,0,58.19375,0.0


In [24]:
from sklearn.preprocessing import StandardScaler

numerical_cols = df.select_dtypes(include='number').columns.tolist()
exclude_cols = ['Status_encoded'] + [col for col in df.columns if col.startswith('Year_')] + ['Country_encoded']
numerical_final = [col for col in numerical_cols if col not in exclude_cols]
df[numerical_final] = StandardScaler().fit_transform(df[numerical_final])

In [25]:
df.head()

,Life expectancy,Adult Mortality,Alcohol,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,...,Year_2006,Year_2007,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015
0,-0.444448,0.791586,-1.172958,-0.705861,-0.110384,-0.964715,0.255359,-3.278638,0.925806,-0.732952,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.980950,0.856072,-1.172958,-0.838704,-0.168124,-0.989810,0.274060,-1.051482,0.934140,-0.859877,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.980950,0.831890,-1.172958,-0.750142,-0.173531,-1.014905,0.292761,-0.880163,0.913306,-0.775260,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,-1.023029,0.864132,-1.172958,-0.617299,0.032045,-1.040000,0.317696,-0.666013,1.075815,-0.648335,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-1.054587,0.888314,-1.172958,-0.573018,0.051757,-1.060076,0.342631,-0.623183,0.804966,-0.606027,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [27]:
df.to_csv("life_expectancy_scaled.csv", index=False)